In [2]:
import numpy as np
from datetime import datetime
from sklearn.model_selection import KFold
import pandas as pd
import keras
from keras.models import Model, load_model
from keras.layers import Dense,Input,GRU,Dropout,concatenate,Permute,Reshape,Lambda,RepeatVector,merge,MaxPooling1D,Embedding,Activation,Conv1D,Flatten
import pickle
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.initializers import RandomUniform
from keras.optimizers import *
from sklearn.metrics import roc_auc_score, mean_squared_error,roc_curve
from keras.backend import eval
from keras import backend as K

Using TensorFlow backend.


In [3]:
learning_rate = 0.001
n_hidden = 200
drop_out = 0.2
time_length = 365
n_feature = 2070
n_emb = 100

In [1]:
def get_rnn_model():
    X_input = Input(shape=(time_length,n_feature), name='x_main', dtype='float32')
    x_dmgs = Input(shape=(3,), name='x_dmgs', dtype='float32')
    
    gru_h = GRU(n_hidden,activation='tanh', recurrent_activation='sigmoid', 
                use_bias=True, kernel_initializer='glorot_uniform', 
                recurrent_initializer='orthogonal', bias_initializer='zeros')(X_input)
    gru_h = Dropout(drop_out)(gru_h)
    comb = concatenate([gru_h,x_dmgs])
    h_t = Dense(1,activation='sigmoid')(comb)
    
    
    model = Model(inputs=[X_input, x_dmgs], outputs = h_t)
    
    opt = keras.optimizers.adam(lr=learning_rate)

    
    model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])

    return model

In [19]:
def get_cnn_model():
    X_input = Input(shape=(time_length,n_feature), name='x_main', dtype='float32')
    x_dmgs = Input(shape=(3,), name='x_dmgs', dtype='float32')
       
    preds = Conv1D(3,16,activation='relu')(X_input)
    preds = BatchNormalization()(preds)
    preds = Dropout(0.5)(preds)
    preds = MaxPooling1D(pool_size=2)(preds)
    preds = BatchNormalization()(preds)
    preds = Flatten()(preds)
    preds = Dropout(0.5)(preds)
    comb = concatenate([preds,x_dmgs])
    preds = Dense(32,activation='relu')(comb)
    preds = Dropout(0.5)(preds)
    preds = Dense(1,activation='sigmoid')(preds)

    model = Model(inputs=[X_input, x_dmgs], outputs=preds)

    # initiate RMSprop optimizer
    opt = keras.optimizers.adam(lr=learning_rate)

    # Let's train the model using RMSprop
    model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])

    return model

In [6]:
def get_pooling_model(ptype):
    X_input = Input(shape=(time_length,n_feature), name='x_main', dtype='float32')
    x_dmgs = Input(shape=(3,), name='x_dmgs', dtype='float32')
    
    gru_h = GRU(n_hidden,activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform', 
                recurrent_initializer='orthogonal', bias_initializer='zeros',return_sequences=True)(X_input)
    
    gru_out = Lambda(lambda x: x[:,-1,:])(gru_h)
    gru_out = Reshape((n_hidden,))(gru_out)
    
    if ptype == 'max':
        max_gru = MaxPooling1D(pool_size=time_length)(gru_h)
        max_gru = Reshape((n_hidden,))(max_gru)
        gru_h = concatenate([gru_out,max_gru])
    
    elif ptype == 'avg':
        avg_gru = Lambda(lambda x: K.mean(x,axis=-2))(gru_h)
        avg_gru = Reshape((n_hidden,))(avg_gru)
        gru_h = concatenate([gru_out,avg_gru])
    
    elif ptype == 'min':
        min_gru = Lambda(lambda x: -x)(gru_h)
        min_gru = MaxPooling1D(pool_size=time_length)(min_gru)
        min_gru = Lambda(lambda x: -x)(min_gru)
        min_gru = Reshape((n_hidden,))(min_gru)
        gru_h = concatenate([gru_out,min_gru])
        
    elif ptype == 'bpv':
        max_gru = MaxPooling1D(pool_size=time_length)(gru_h)
        max_gru = Reshape((n_hidden,))(max_gru)
        avg_gru = Lambda(lambda x: K.mean(x,axis=-2))(gru_h)
        avg_gru = Reshape((n_hidden,))(avg_gru)
        min_gru = Lambda(lambda x: -x)(gru_h)
        min_gru = MaxPooling1D(pool_size=time_length)(min_gru)
        min_gru = Lambda(lambda x: -x)(min_gru)
        min_gru = Reshape((n_hidden,))(min_gru)
        gru_h = concatenate([gru_out,max_gru,avg_gru,min_gru])
        
    elif ptype == 'maxmin':
        max_gru = MaxPooling1D(pool_size=time_length)(gru_h)
        max_gru = Reshape((n_hidden,))(max_gru)
        min_gru = Lambda(lambda x: -x)(gru_h)
        min_gru = MaxPooling1D(pool_size=time_length)(min_gru)
        min_gru = Lambda(lambda x: -x)(min_gru)
        min_gru = Reshape((n_hidden,))(min_gru)
        gru_h = concatenate([gru_out,max_gru,min_gru])
        
    #gru_h = concatenate([gru_out,max_gru,avg_gru])
    gru_h = Dropout(drop_out)(gru_h)
    
    comb = concatenate([gru_h,x_dmgs])
    h_t = Dense(1,activation='sigmoid')(comb)
    
    
    model = Model(inputs=[X_input, x_dmgs], outputs = h_t)
    
    opt = keras.optimizers.adam(lr=learning_rate)

    model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])

    return model

In [ ]:
bsize = 250
max_epoch = 6
model_folder0 = 'one_year_prediction/cnn/'
model_folder1 = 'one_year_prediction/rnn-mh/'
model_folder2 = 'one_year_prediction/rnn-mh-bpv/'
model_folder3 = 'one_year_prediction/rnn-mh-maxmin/'
!mkdir $model_folder0
!mkdir $model_folder1
!mkdir $model_folder2
!mkdir $model_folder3

kf = KFold(n_splits=5, shuffle=False)
cv_counter = 0
with open(model_folder0+'cv_results.txt','w') as out0:
    with open(model_folder1+'cv_results.txt','w') as out1:
        with open(model_folder2+'cv_results.txt','w') as out2:
            with open(model_folder3+'cv_results.txt','w') as out3:
                for train_index, test_index in kf.split(np.arange(54)):
                    cv_counter += 1
                    if cv_counter > 2:
                        print(test_index)
                        out0.write("##".join(['cv order',str(cv_counter),'leave out index',str(test_index)])+'\n\n')
                        out1.write("##".join(['cv order',str(cv_counter),'leave out index',str(test_index)])+'\n\n')
                        out2.write("##".join(['cv order',str(cv_counter),'leave out index',str(test_index)])+'\n\n')
                        out3.write("##".join(['cv order',str(cv_counter),'leave out index',str(test_index)])+'\n\n')
                        model0 = get_cnn_model()
                        model1 = get_rnn_model()
                        model2 = get_pooling_model(ptype='bpv')
                        model3 = get_pooling_model(ptype='maxmin')
                        val_loss_list0,val_loss_list1,val_loss_list2,val_loss_list3 = [],[],[],[]

                        for k in range(max_epoch):
                            train_loss0,train_loss1,train_loss2,train_loss3 = np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0])
                            val_loss0,val_loss1,val_loss2,val_loss3 = np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0])
                            y_test_all0,y_test_all1,y_test_all2,y_test_all3 = None,None,None,None
                            y_pred_all0,y_pred_all1,y_pred_all2,y_pred_all3 = None,None,None,None

                            for i in train_index:
                                loadata = np.load('0702data'+str(i)+'.npz')
                                dmg_data = np.load('additional_fields/0317data'+str(i)+'_additionalFields.npz')
                                x_dmg=dmg_data['values']
                                if i == 53:
                                    dmg_53 = np.zeros((9000,4))
                                    for mm in range(9000):
                                        dmg_53[mm,:] = x_dmg[mm]
                                    x_dmg = dmg_53
                                x_dmg[:,2] = abs(x_dmg[:,3]-2)
                                x_dmg = x_dmg[:,:3]
                                x = loadata['InputX3D']
                                y = loadata['Output3D']
                                ### use this when running the model with only one year data
                                ### x = x[:,365:,:]
                                x=np.concatenate((x[:,:,:505],x[:,:,539:]),axis=2)
                                for j in range(0,x_dmg.shape[0],bsize):
                                    batch_loss0 = model0.train_on_batch({'x_main':x[j:j+bsize,:,:],'x_dmgs':x_dmg[j:j+bsize]}, y[j:j+bsize,0])
                                    train_loss0 = np.add(train_loss0,batch_loss0)
                                    batch_loss1 = model1.train_on_batch({'x_main':x[j:j+bsize,:,:],'x_dmgs':x_dmg[j:j+bsize]}, y[j:j+bsize,0])
                                    train_loss1 = np.add(train_loss1,batch_loss1)
                                    batch_loss2 = model2.train_on_batch({'x_main':x[j:j+bsize,:,:],'x_dmgs':x_dmg[j:j+bsize]}, y[j:j+bsize,0])
                                    train_loss2 = np.add(train_loss2,batch_loss2)
                                    batch_loss3 = model3.train_on_batch({'x_main':x[j:j+bsize,:,:],'x_dmgs':x_dmg[j:j+bsize]}, y[j:j+bsize,0])
                                    train_loss3 = np.add(train_loss3,batch_loss3)
                                print(i, datetime.now())

                            model0.save(model_folder0+'cv_model_'+str(cv_counter)+'_'+str(k)+'.h5')
                            model1.save(model_folder1+'cv_model_'+str(cv_counter)+'_'+str(k)+'.h5')
                            model2.save(model_folder2+'cv_model_'+str(cv_counter)+'_'+str(k)+'.h5')
                            model3.save(model_folder3+'cv_model_'+str(cv_counter)+'_'+str(k)+'.h5')
                            train_loss0 = np.divide(train_loss0,(len(train_index)*10000/bsize))
                            train_loss1 = np.divide(train_loss1,(len(train_index)*10000/bsize))
                            train_loss2 = np.divide(train_loss2,(len(train_index)*10000/bsize))
                            train_loss3 = np.divide(train_loss3,(len(train_index)*10000/bsize))
                            print('model0:','epoch',k,'train_loss:',train_loss0)
                            print('model1:','epoch',k,'train_loss:',train_loss1)
                            print('model2:','epoch',k,'train_loss:',train_loss2)
                            print('model3:','epoch',k,'train_loss:',train_loss3)
                            out0.write("\t".join([str(datetime.now()),'epoch',str(k),'train loss',str(train_loss0)])+'\n')
                            out1.write("\t".join([str(datetime.now()),'epoch',str(k),'train loss',str(train_loss1)])+'\n')
                            out2.write("\t".join([str(datetime.now()),'epoch',str(k),'train loss',str(train_loss2)])+'\n')
                            out3.write("\t".join([str(datetime.now()),'epoch',str(k),'train loss',str(train_loss3)])+'\n')

                            if k > 1:
                                for i in test_index:
                                    loadata = np.load('0702data'+str(i)+'.npz')
                                    x=loadata['InputX3D']
                                    y=loadata['Output3D']
                                    ### use this when running the model with only one year data
                                    ### x = x[:,time_length:,:]
                                    x=np.concatenate((x[:,:,:505],x[:,:,539:]),axis=2)
                                    dmg_data = np.load('additional_fields/0317data'+str(i)+'_additionalFields.npz')
                                    x_val_dmg=dmg_data['values']
                                    if i == 53:
                                        dmg_53 = np.zeros((9000,4))
                                        for mm in range(9000):
                                            dmg_53[mm,:] = x_val_dmg[mm]
                                        x_val_dmg = dmg_53
                                        x = x[:9000]
                                        y = y[:9000]
                                    x_val_dmg[:,2] = abs(x_val_dmg[:,3]-2)
                                    x_val_dmg = x_val_dmg[:,:3]
                                    y_test_all = (y[:,0] if i==test_index[0] else np.append(y_test_all, y[:,0]))
                                    batch_loss0 = model0.evaluate({'x_main':x,'x_dmgs':x_val_dmg},y[:,0])
                                    batch_loss1 = model1.evaluate({'x_main':x,'x_dmgs':x_val_dmg},y[:,0])
                                    batch_loss2 = model2.evaluate({'x_main':x,'x_dmgs':x_val_dmg},y[:,0])
                                    batch_loss3 = model3.evaluate({'x_main':x,'x_dmgs':x_val_dmg},y[:,0])
                                    val_loss0 = np.add(val_loss0,batch_loss0)
                                    val_loss1 = np.add(val_loss1,batch_loss1)
                                    val_loss2 = np.add(val_loss2,batch_loss2)
                                    val_loss3 = np.add(val_loss3,batch_loss3)
                                    print(i, datetime.now())
                                    y_pred0 = model0.predict({'x_main':x,'x_dmgs':x_val_dmg})
                                    np.savez_compressed(model_folder0+'pred_y_'+str(cv_counter)+'_'+str(k)+'_'+str(i),y_pred=y_pred0)
                                    y_pred_all0 = (y_pred0 if i==test_index[0] else np.append(y_pred_all0, y_pred0))
                                    y_pred1 = model1.predict({'x_main':x,'x_dmgs':x_val_dmg})
                                    np.savez_compressed(model_folder1+'pred_y_'+str(cv_counter)+'_'+str(k)+'_'+str(i),y_pred=y_pred1)
                                    y_pred_all1 = (y_pred1 if i==test_index[0] else np.append(y_pred_all1, y_pred1))
                                    y_pred2 = model2.predict({'x_main':x,'x_dmgs':x_val_dmg})
                                    np.savez_compressed(model_folder2+'pred_y_'+str(cv_counter)+'_'+str(k)+'_'+str(i),y_pred=y_pred2)
                                    y_pred_all2 = (y_pred2 if i==test_index[0] else np.append(y_pred_all2, y_pred2))
                                    y_pred3 = model3.predict({'x_main':x,'x_dmgs':x_val_dmg})
                                    np.savez_compressed(model_folder3+'pred_y_'+str(cv_counter)+'_'+str(k)+'_'+str(i),y_pred=y_pred3)
                                    y_pred_all3 = (y_pred3 if i==test_index[0] else np.append(y_pred_all3, y_pred3))

                                val_loss0 = np.divide(val_loss0,len(test_index))
                                val_loss_list0.append(val_loss0[0])
                                pred_auc0 = roc_auc_score(y_test_all, y_pred_all0)
                                val_loss1 = np.divide(val_loss1,len(test_index))
                                val_loss_list1.append(val_loss1[0])
                                pred_auc1 = roc_auc_score(y_test_all, y_pred_all1)
                                val_loss2 = np.divide(val_loss2,len(test_index))
                                val_loss_list2.append(val_loss2[0])
                                pred_auc2 = roc_auc_score(y_test_all, y_pred_all2)
                                val_loss3 = np.divide(val_loss3,len(test_index))
                                val_loss_list3.append(val_loss3[0])
                                pred_auc3 = roc_auc_score(y_test_all, y_pred_all3)
                                print('model0: ','epoch',k,'validation loss',val_loss0,'validation auc',pred_auc0)
                                print('model1: ','epoch',k,'validation loss',val_loss1,'validation auc',pred_auc1)
                                print('model2: ','epoch',k,'validation loss',val_loss2,'validation auc',pred_auc2)
                                print('model3: ','epoch',k,'validation loss',val_loss3,'validation auc',pred_auc3)
                                out0.write("\t".join([str(datetime.now()),'epoch',str(k),'validation loss',str(val_loss0),'val_auc',str(pred_auc0)])+'\n')
                                out1.write("\t".join([str(datetime.now()),'epoch',str(k),'validation loss',str(val_loss1),'val_auc',str(pred_auc1)])+'\n')
                                out2.write("\t".join([str(datetime.now()),'epoch',str(k),'validation loss',str(val_loss2),'val_auc',str(pred_auc2)])+'\n')
                                out3.write("\t".join([str(datetime.now()),'epoch',str(k),'validation loss',str(val_loss3),'val_auc',str(pred_auc3)])+'\n')

mkdir: cannot create directory ‘one_year_prediction/cnn/’: File exists
mkdir: cannot create directory ‘one_year_prediction/rnn-mh/’: File exists
mkdir: cannot create directory ‘one_year_prediction/rnn-mh-bpv/’: File exists
mkdir: cannot create directory ‘one_year_prediction/rnn-mh-maxmin/’: File exists
[ 0  1  2  3  4  5  6  7  8  9 10]
11 2018-08-01 01:03:11.965982
12 2018-08-01 01:09:34.030762


In [ ]:
bsize = 250
max_epoch = 4
model_folder0 = 'one_year_prediction/rnn_mh/'

!mkdir $model_folder0


kf = KFold(n_splits=5, shuffle=False)
cv_counter = 0
with open(model_folder0+'cv_results.txt','a') as out0:
    for train_index, test_index in kf.split(np.arange(54)):
        cv_counter += 1
        if cv_counter > 3:
            print(test_index)
            out0.write("##".join(['cv order',str(cv_counter),'leave out index',str(test_index)])+'\n\n')
            model0 = get_rnn_model()
            val_loss_list0,val_loss_list1,val_loss_list2,val_loss_list3 = [],[],[],[]

            for k in range(max_epoch):
                train_loss0,train_loss1,train_loss2,train_loss3 = np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0])
                val_loss0,val_loss1,val_loss2,val_loss3 = np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0]),np.array([0.0,0.0])
                y_test_all0,y_test_all1,y_test_all2,y_test_all3 = None,None,None,None
                y_pred_all0,y_pred_all1,y_pred_all2,y_pred_all3 = None,None,None,None

                for i in train_index:
                    loadata = np.load('0702data'+str(i)+'.npz')
                    dmg_data = np.load('additional_fields/0317data'+str(i)+'_additionalFields.npz')
                    x_dmg=dmg_data['values']
                    if i == 53:
                        dmg_53 = np.zeros((9000,4))
                        for mm in range(9000):
                            dmg_53[mm,:] = x_dmg[mm]
                        x_dmg = dmg_53
                    x_dmg[:,2] = abs(x_dmg[:,3]-2)
                    x_dmg = x_dmg[:,:3]
                    x = loadata['InputX3D']
                    y = loadata['Output3D']
                    ### use this when running the model with only one year data
                    ### x = x[:,time_length:,:]
                    x=np.concatenate((x[:,:,:505],x[:,:,539:]),axis=2)
                    for j in range(0,x_dmg.shape[0],bsize):
                        batch_loss0 = model0.train_on_batch({'x_main':x[j:j+bsize,:,:],'x_dmgs':x_dmg[j:j+bsize]}, y[j:j+bsize,0])
                        train_loss0 = np.add(train_loss0,batch_loss0)
                    print(i, datetime.now())

                model0.save(model_folder0+'cv_model_'+str(cv_counter)+'_'+str(k)+'.h5')
                
                train_loss0 = np.divide(train_loss0,(len(train_index)*10000/bsize))
                
                print('model0:','epoch',k,'train_loss:',train_loss0)
                
                out0.write("\t".join([str(datetime.now()),'epoch',str(k),'train loss',str(train_loss0)])+'\n')
                

                if k > 1:
                    for i in test_index:
                        loadata = np.load('0702data'+str(i)+'.npz')
                        x=loadata['InputX3D']
                        y=loadata['Output3D']
                        ### use this when running the model with only one year data
                        ### x = x[:,time_length:,:]
                        x=np.concatenate((x[:,:,:505],x[:,:,539:]),axis=2)
                        dmg_data = np.load('additional_fields/0317data'+str(i)+'_additionalFields.npz')
                        x_val_dmg=dmg_data['values']
                        if i == 53:
                            dmg_53 = np.zeros((9000,4))
                            for mm in range(9000):
                                dmg_53[mm,:] = x_val_dmg[mm]
                            x_val_dmg = dmg_53
                            x = x[:9000]
                            y = y[:9000]
                        x_val_dmg[:,2] = abs(x_val_dmg[:,3]-2)
                        x_val_dmg = x_val_dmg[:,:3]
                        y_test_all = (y[:,0] if i==test_index[0] else np.append(y_test_all, y[:,0]))
                        batch_loss0 = model0.evaluate({'x_main':x,'x_dmgs':x_val_dmg},y[:,0])
                        
                        val_loss0 = np.add(val_loss0,batch_loss0)
                       
                        print(i, datetime.now())
                        y_pred0 = model0.predict({'x_main':x,'x_dmgs':x_val_dmg})
                        np.savez_compressed(model_folder0+'pred_y_'+str(cv_counter)+'_'+str(k)+'_'+str(i),y_pred=y_pred0)
                        y_pred_all0 = (y_pred0 if i==test_index[0] else np.append(y_pred_all0, y_pred0))
                       
                    val_loss0 = np.divide(val_loss0,len(test_index))
                    val_loss_list0.append(val_loss0[0])
                    pred_auc0 = roc_auc_score(y_test_all, y_pred_all0)
                    
                    print('model0: ','epoch',k,'validation loss',val_loss0,'validation auc',pred_auc0)
                   
                    out0.write("\t".join([str(datetime.now()),'epoch',str(k),'validation loss',str(val_loss0),'val_auc',str(pred_auc0)])+'\n')
                    

mkdir: cannot create directory ‘one_year_prediction/rnn_mh/’: File exists
[33 34 35 36 37 38 39 40 41 42 43]
0 2018-08-04 11:18:44.979755
1 2018-08-04 11:19:43.327630
2 2018-08-04 11:20:42.215536
3 2018-08-04 11:21:41.235919
4 2018-08-04 11:22:39.604062
5 2018-08-04 11:23:38.311938
6 2018-08-04 11:24:36.784239
7 2018-08-04 11:25:35.698139
8 2018-08-04 11:26:34.257972
9 2018-08-04 11:27:33.899851
10 2018-08-04 11:28:32.824257
11 2018-08-04 11:29:31.962498
12 2018-08-04 11:30:30.906894
13 2018-08-04 11:31:30.081898
14 2018-08-04 11:32:28.658309
15 2018-08-04 11:33:27.274018
16 2018-08-04 11:34:26.301747
17 2018-08-04 11:35:24.832549
18 2018-08-04 11:36:23.852417
19 2018-08-04 11:37:23.016510
20 2018-08-04 11:38:22.006359
21 2018-08-04 11:39:20.594635
22 2018-08-04 11:40:19.401763
23 2018-08-04 11:41:17.606612
24 2018-08-04 11:42:16.683735
25 2018-08-04 11:43:15.373837
26 2018-08-04 11:44:14.242277
27 2018-08-04 11:45:13.084487
28 2018-08-04 11:46:11.431142
29 2018-08-04 11:47:10.676420
3